In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm
import numpy as np
import arpa

import pynlpl.lm.lm as pyn
import pynlpl.lm.srilm as pynLM

import torch.nn as nn
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

GeForce GTX 960M
True


/usr/local/lib/python3.6/dist-packages/pynlpl/lm/srilm.py:25: UserWarning: srilmcc module is not compiled
  warnings.warn("srilmcc module is not compiled")


# Load NNs

In [2]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        #self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)

        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        #x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x)) 
        x = torch.sigmoid(self.fc4(x)) 
        return x
    
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(5, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 1)

        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x)) 
        return x   

net1 = Net1().cuda()
net1.load_state_dict(torch.load('NN saves/4431_lr=0.03_MAE'))
net1.eval()

net2 = Net2().cuda()
net2.load_state_dict(torch.load('NN saves/5541_NN'))
net2.eval()

Net2(
  (fc1): Linear(in_features=5, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=1, bias=True)
)

# Load in LM

In [3]:
#lm = arpa.loadf('../../rsc/smoothedLM.arpa')[0]
lm = pyn.ARPALanguageModel('../../rsc/smoothedLM.arpa')
#lm = pynLM.SRILM('../../rsc/smoothedLM.arpa', 1)


SRILMException: SRILM is not downloaded and compiled.Please follow the instructions in makesrilmcc

# LM probabilities

In [14]:
#r = 1

print('Seen:\nPgt(POSTS | JOBS, FOR) = ' + str(lm.p("JOBS FOR POSTS")))


print('\nUnseen:\nPgt(MONEY | JOBS, FOR) = ' + str(lm.p("JOBS FOR MONEY")))
print('Pgt(CASH | JOBS, FOR) = ' + str(lm.p("JOBS FOR CASH")))


Seen:
 Pgt(POSTS | JOBS, FOR) = 5.6950009387085187e-05

Unseen:
Pgt(MONEY | JOBS, FOR) = 0.0019743529819056757
Pgt(CASH | JOBS, FOR) = 0.0019743529819056757


In [9]:
#r = 1

print('Seen:\nPgt(POSTS | JOBS, FOR) = ' + str(lm.scoreword("JOBS FOR POSTS")))


print('\nUnseen:\nPgt(MONEY | JOBS, FOR) = ' + str(lm.prob("JOBS FOR HOS")))
print('Pgt(CASH | JOBS, FOR) = ' + str(lm.prob("JOBS FOR CASH")))



Seen:
Pgt(POSTS | JOBS, FOR) = -8.790399507886116


AttributeError: 'ARPALanguageModel' object has no attribute 'prob'

# Counts:

In [ ]:
#TRIGRAM
# JOBS FOR MONEY - 0
# JOBS FOR POSTS - 2
# JOBS FOR CASH - 1

#PREFIX
# JOBS FOR - 8

#(N-1)-GRAM
#FOR MONEY - 11
#FOR CASH - 1
#FOR POSTS - 2

#(N-2)-GRAM
# MONEY - 474
# CASH - 125
# POSTS - 83

#PRE-PREFIX
# JOBS - 147

# NN results:

In [23]:
#prefix - trigram - (n-1)-gram - (n-2)-gram - pre-prefix

#jobs for posts
inpt = torch.tensor([[8 , 1, 2 , 83 ]],device=device).float()
inpt = 1/inpt
out = float(net1(inpt))
#print(float(out))

print('NN discounted prob: ' + str(out))
print('Discounted prob from test set: ' + str(0.3*1/8))



inpt = torch.tensor([[8 , 1, 2 , 83 , 147]],device=device).float()
inpt = 1/inpt
out = float(net2(inpt))
#print(float(out))

print('\nNN \'true\' MLE: ' + str(out))
print('MLE from test set: ' + str(1/8))


NN discounted prob: 0.065711610019207
Discounted prob from test set: 0.0375

NN 'true' MLE: 0.15154613554477692
MLE from test set: 0.125


In [17]:
#prefix - trigram - (n-1)-gram - (n-2)-gram - pre-prefix

#jobs for cash
inpt = torch.tensor([[8 , 1, 1 , 125 ]],device=device).float()
inpt = 1/inpt
out = float(net1(inpt))
#print(float(out))

print('NN discounted prob: ' + str(out))
print('Discounted prob from test set: ' + str(0.3*1/8))



inpt = torch.tensor([[8 , 1, 1 , 125 , 147]],device=device).float()
inpt = 1/inpt
out = float(net2(inpt))
#print(float(out))

print('\nNN \'true\' MLE: ' + str(out))
print('MLE from test set: ' + str(1/8))


NN discounted prob: 0.06568172574043274
Discounted prob from test set: 0.0375

NN 'true' MLE: 0.1473076343536377
MLE from test set: 0.125


In [22]:
#prefix - trigram - (n-1)-gram - (n-2)-gram - pre-prefix

#jobs for money
inpt = torch.tensor([[8, 0, 11 , 474 ]],device=device).float()
inpt = 1/inpt
out = float(net1(inpt))
#print(float(out))

print('NN discounted prob: ' + str(out))
print('Discounted prob from test set: ' + str(0/8))



inpt = torch.tensor([[8 , 0, 11 , 474 , 147]],device=device).float()
inpt = 1/inpt
out = float(net2(inpt))
#print(float(out))

print('\nNN \'true\' MLE: ' + str(out))
print('MLE from test set: ' + str(0/8))


NN discounted prob: 6.656724167442007e-07
Discounted prob from test set: 0.0

NN 'true' MLE: 2.4396895241807215e-06
MLE from test set: 0.0
